<a href="https://colab.research.google.com/github/happyhackerhour/happyhackerhour/blob/main/ngrok-squid-proxy-tricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mount your google drive & symlink it to `/drive`

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh

Mounted at /content/drive


#(optional) If a TPU is attached, print its address

This notebook works for all runtime types: CPU, GPU, and TPU.

If connected to TPU runtime, this cell prints info about the attached TPU. (This is just for convenience / informational purposes.)

In [ ]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('Not connected to a TPU runtime')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)
  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
  print('TPU devices:')
  pprint.pprint(devices)

TPU address is grpc://10.44.120.90:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 1173016086274077236),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2991867235668810802),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 6184859761337604570),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 2958198024988600890),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 7018531718559331614),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 4577201967883118425),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 1807161912338548618),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 6466936639391228903),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 5609808055865915

# Install `sshd`, `tmux`, and `vim`

You can add any other prerequisites you want, e.g. `emacs`. That way, when you ssh into the colab instance, all of these will already be installed.

(You'd otherwise have to run `apt-get install emacs` every time you ssh into a fresh colab instance. It's easier just to install all your favorite programs here, ahead of time.)

In [2]:
!apt-get install ssh tmux vim

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tmux is already the newest version (2.6-3ubuntu0.2).
The following additional packages will be installed:
  libgpm2 ncurses-term openssh-server openssh-sftp-server python3-certifi
  python3-chardet python3-idna python3-pkg-resources python3-requests
  python3-six python3-urllib3 ssh-import-id vim-common vim-runtime xxd
Suggested packages:
  gpm molly-guard monkeysphere rssh ssh-askpass ufw python3-setuptools
  python3-cryptography python3-openssl python3-socks ctags vim-doc vim-scripts
The following NEW packages will be installed:
  libgpm2 ncurses-term openssh-server openssh-sftp-server python3-certifi
  python3-chardet python3-idna python3-pkg-resources python3-requests
  python3-six python3-urllib3 ssh ssh-import-id vim vim-common vim-runtime xxd
0 upgraded, 17 newly installed, 0 to remove and 39 not upgraded.
Need to get 7,876 kB of archives.
After this operation, 40.3 MB of additional d

# Create an sshd configuration file

- Normally, you shouldn't need to modify this next cell; the configuration is designed to work with colab.

Simply run this cell:

In [4]:
#@title
%%writefile /drive/ngrok-ssh/sshd_config
#	$OpenBSD: sshd_config,v 1.101 2017/03/14 07:19:07 djm Exp $

# This is the sshd server system-wide configuration file.  See
# sshd_config(5) for more information.

# This sshd was compiled with PATH=/usr/bin:/bin:/usr/sbin:/sbin

# The strategy used for options in the default sshd_config shipped with
# OpenSSH is to specify options with their default value where
# possible, but leave them commented.  Uncommented options override the
# default value.

#Port 22
#AddressFamily any
#ListenAddress 0.0.0.0
#ListenAddress ::

#HostKey /etc/ssh/ssh_host_rsa_key
#HostKey /etc/ssh/ssh_host_ecdsa_key
#HostKey /etc/ssh/ssh_host_ed25519_key

# Ciphers and keying
#RekeyLimit default none

# Logging
#SyslogFacility AUTH
#LogLevel INFO

# Authentication:

#LoginGraceTime 2m
#PermitRootLogin prohibit-password
#StrictModes yes
#MaxAuthTries 6
#MaxSessions 10

#PubkeyAuthentication yes

# Expect .ssh/authorized_keys2 to be disregarded by default in future.
#AuthorizedKeysFile	.ssh/authorized_keys .ssh/authorized_keys2

#AuthorizedPrincipalsFile none

#AuthorizedKeysCommand none
#AuthorizedKeysCommandUser nobody

# For this to work you will also need host keys in /etc/ssh/ssh_known_hosts
#HostbasedAuthentication no
# Change to yes if you don't trust ~/.ssh/known_hosts for
# HostbasedAuthentication
#IgnoreUserKnownHosts no
# Don't read the user's ~/.rhosts and ~/.shosts files
#IgnoreRhosts yes

# To disable tunneled clear text passwords, change to no here!
#PasswordAuthentication yes
#PermitEmptyPasswords no

# Change to yes to enable challenge-response passwords (beware issues with
# some PAM modules and threads)
ChallengeResponseAuthentication no

# Kerberos options
#KerberosAuthentication no
#KerberosOrLocalPasswd yes
#KerberosTicketCleanup yes
#KerberosGetAFSToken no

# GSSAPI options
#GSSAPIAuthentication no
#GSSAPICleanupCredentials yes
#GSSAPIStrictAcceptorCheck yes
#GSSAPIKeyExchange no

# Set this to 'yes' to enable PAM authentication, account processing,
# and session processing. If this is enabled, PAM authentication will
# be allowed through the ChallengeResponseAuthentication and
# PasswordAuthentication.  Depending on your PAM configuration,
# PAM authentication via ChallengeResponseAuthentication may bypass
# the setting of "PermitRootLogin without-password".
# If you just want the PAM account and session checks to run without
# PAM authentication, then enable this but set PasswordAuthentication
# and ChallengeResponseAuthentication to 'no'.
UsePAM yes

#AllowAgentForwarding yes
AllowTcpForwarding yes
#GatewayPorts no
X11Forwarding yes
#X11DisplayOffset 10
#X11UseLocalhost yes
#PermitTTY yes
PrintMotd no
#PrintLastLog yes
#TCPKeepAlive yes
#UseLogin no
#PermitUserEnvironment no
#Compression delayed
#ClientAliveInterval 0
#ClientAliveCountMax 3
#UseDNS no
#PidFile /var/run/sshd.pid
#MaxStartups 10:30:100
#PermitTunnel no
#ChrootDirectory none
#VersionAddendum none

# no default banner path
#Banner none

# Allow client to pass locale environment variables
AcceptEnv LANG LC_*

# override default of no subsystems
Subsystem	sftp	/usr/lib/openssh/sftp-server

# Example of overriding settings on a per-user basis
#Match User anoncvs
#	X11Forwarding no
#	AllowTcpForwarding no
#	PermitTTY no
#	ForceCommand cvs server

Overwriting /drive/ngrok-ssh/sshd_config


# Paste the contents of your local `~/.ssh/id_rsa.pub` file into this next cell

- If you don't have an `~/.ssh/id_rsa.pub` file, use `ssh-keygen` to make one.

- Remember to actually run the cell after you paste your key into it!



In [5]:
%%writefile ~/.ssh/authorized_keys
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQCYjSdCAj4AQa5Q/YSIn15sPP+l6HMqLJD3HBhbus1XXP9t48F0UZLdf5iwvrxgzHCpuEkOveyQcr2Uo8+XaDDsTj7ouE0GDsEAVxD0NauBMBa7QYfL3a5XsBsSVoEOhtkgiWN+FkkGJejt6ugmhgYEbtqEDYOVgP/SZOYSaXxb0tGgWCLXYMbK9xazIP1ZLMEiFj4fTkbZOMTx6H+SHre8fzT3ITqP27bnViMxG0AroZPLJvOHDq/oHrl6de2szzu2VitS1oulmBJIFlWdiWTkd1dk4vBDqJNKEfKdcm+A1lJIUpOkqcNnSMuLD1mjmFtgy96GfShhjkqV64ImOjXV0OimXAHoyiXaa3IF9pgOiXuIjDIDSpyfJLmxad1uhp5IIH7khkCwWpB+XEaQy9bz0GjkWDrAO26GNAtnyZezyknHLJmVtId3nGcabNm3IxLYPkRSBNRkPG21HF7bThkMMi76EoEfiMpTbwj9TKatLbt+9WCjIJkeDV+1bahwpV0= chronos@localhost

Writing /root/.ssh/authorized_keys


# Restart sshd

In [6]:
!cp /drive/ngrok-ssh/sshd_config /etc/ssh/sshd_config
!service ssh restart
!mkdir -p ~/.ssh

 * Restarting OpenBSD Secure Shell server sshd
   ...done.


#Install ngrok

In [7]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2021-07-12 03:44:14--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.84.56.8, 52.45.133.242, 54.159.229.235, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.84.56.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  32.4MB/s    in 0.4s    

2021-07-12 03:44:15 (32.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


# Sign-in with ngrok

- Sign up for a free ngrok account: https://ngrok.com

- Copy your authtoken from https://dashboard.ngrok.com/auth and paste it into the following cell

- Run the cell

In [8]:
!/ngrok authtoken 1lKdLflwF27NWuwptI1ld2KZ7J4_3bFGEDT7YLTaCq1gPAjnL

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Create various ngrok tunnel configurations

Run each of these cells, one by one:

In [11]:
%%writefile /drive/ngrok-ssh/ssh.yml
tunnels:
  ssh:
    proto: tcp
    addr: 22

Overwriting /drive/ngrok-ssh/ssh.yml


In [12]:
%%writefile /drive/ngrok-ssh/squid.yml
tunnels:
  squid:
    proto: tcp
    addr: 3128
    #inspect: false
    #bind_tls: true

Overwriting /drive/ngrok-ssh/squid.yml


In [13]:
%%writefile /drive/ngrok-ssh/http8080.yml
tunnels:
  http8080:
    proto: http
    addr: 8080
    inspect: false
    bind_tls: true

Writing /drive/ngrok-ssh/http8080.yml


In [14]:
%%writefile /drive/ngrok-ssh/tcp8080.yml
tunnels:
  tcp8080:
    proto: tcp
    addr: 8080

Writing /drive/ngrok-ssh/tcp8080.yml


In [15]:
import os
if 'COLAB_TPU_ADDR' in os.environ:
  with open('/drive/ngrok-ssh/tpu.yml', 'w') as f:
    f.write("""
tunnels:
  tpu:
    proto: tcp
    addr: {}
""".format(os.environ['COLAB_TPU_ADDR']).strip())
  print('Wrote /drive/ngrok-ssh/tpu.yml')

Wrote /drive/ngrok-ssh/tpu.yml


In [38]:
%%writefile /drive/ngrok-ssh/run_ngrok.sh
#!/bin/sh
set -x
/ngrok start --config ~/.ngrok2/ngrok.yml --config /drive/ngrok-ssh/ssh.yml --log=stdout "$@"

Overwriting /drive/ngrok-ssh/run_ngrok.sh


In [40]:
!sudo killall ngrok


#Install Squid, Start it and ngrok

In [61]:
get_ipython().system_raw('bash /drive/ngrok-ssh/run_ngrok.sh ssh &')

In [62]:
!ps aux | grep ngrok

root        3176  0.0  0.0  39196  6604 ?        S    04:10   0:00 bash /drive/ngrok-ssh/run_ngrok.sh ssh
root        3177  5.7  0.1 726652 24804 ?        Sl   04:10   0:00 /ngrok start --config /root/.ngrok2/ngrok.yml --config /drive/ngrok-ssh/ssh.yml --log=stdout ssh
root        3187  0.0  0.0  39200  6536 ?        S    04:10   0:00 /bin/bash -c ps aux | grep ngrok
root        3189  0.0  0.0  38576  5668 ?        S    04:10   0:00 grep ngrok


# Make an ngrok API
Let's define some helper functions:

- `get_ngrok_info()` - returns a JSON object containing info about all open ngrok tunnels

- `get_ngrok_tunnels()` - enumerate over all open ngrok tunnels. Yields `name, info` tuples.

- `get_ngrok_tunnel(name)` - get info about a tunnel

- `get_ngrok_url(name, local=False)` - get a tunnel's URL. If `local` is `False`, returns a public URL that anyone can access, otherwise returns the local url being forwarded

In [63]:
import requests
import urllib.parse

def get_ngrok_info():
  return requests.get('http://localhost:4040/api/tunnels').json()

def get_ngrok_tunnels():
  for tunnel in get_ngrok_info()['tunnels']:
    name = tunnel['name']
    yield name, tunnel

def get_ngrok_tunnel(name):
  for name1, tunnel in get_ngrok_tunnels():
    if name == name1:
      return tunnel

def get_ngrok_url(name, local=False):
  if local:
    return get_ngrok_tunnel(name)['config']['addr']
  else:
    return get_ngrok_tunnel(name)['public_url']

In [64]:
get_ngrok_info()

{'tunnels': [{'config': {'addr': 'localhost:22', 'inspect': False},
   'metrics': {'conns': {'count': 0,
     'gauge': 0,
     'p50': 0,
     'p90': 0,
     'p95': 0,
     'p99': 0,
     'rate1': 0,
     'rate15': 0,
     'rate5': 0},
    'http': {'count': 0,
     'p50': 0,
     'p90': 0,
     'p95': 0,
     'p99': 0,
     'rate1': 0,
     'rate15': 0,
     'rate5': 0}},
   'name': 'ssh',
   'proto': 'tcp',
   'public_url': 'tcp://8.tcp.ngrok.io:12712',
   'uri': '/api/tunnels/ssh'}],
 'uri': '/api/tunnels'}

In [56]:
%%writefile /drive/ngrok-ssh/run_ngrok-ssh.sh
#!/bin/sh
set -x
/ngrok tcp 3128

Overwriting /drive/ngrok-ssh/run_ngrok-ssh.sh


In [57]:
!sudo killall ngrok
!bash /drive/ngrok-ssh/run_ngrok-ssh.sh


ngrok: no process found
+ /ngrok tcp 3128




"
^C
^^C


# Skip this section

These are various commands for debugging ngrok.

In [45]:
!bash /drive/ngrok-ssh/run_ngrok.sh ssh

+ /ngrok start --config /root/.ngrok2/ngrok.yml --config /drive/ngrok-ssh/ssh.yml --log=stdout ssh
INFO[07-12|03:59:19] open config file                         path=/root/.ngrok2/ngrok.yml err=nil
INFO[07-12|03:59:19] open config file                         path=/drive/ngrok-ssh/ssh.yml err=nil
t=2021-07-12T03:59:19+0000 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040
t=2021-07-12T03:59:19+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4041
t=2021-07-12T03:59:19+0000 lvl=eror msg="failed to auth" obj=tunnels.session err="Your account 'hviniciusg@gmail.com' is limited to 1 simultaneous ngrok client session.\nActive ngrok client sessions in region 'us':\n  - ts_1vCNrgvwMl60UTpPahEeUR05SFJ (35.225.242.217)\r\n\r\nERR_NGROK_108\r\n"
t=2021-07-12T03:59:19+0000 lvl=eror msg="failed to reconnect session" obj=csess id=ec89746e6fcb err="Your account 'hviniciusg@gmail.com' is limited to 1 simultaneous ngrok client session.\nActive

In [59]:
!curl -s http://localhost:4040/api/tunnels  | python3 -c "import sys, json; url=json.load(sys.stdin)['tunnels'][0]['public_url']; import urllib.parse; print(urllib.parse.urlparse(url).netloc)"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.7/json/__init__.py", line 296, in load
    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)
  File "/usr/lib/python3.7/json/__init__.py", line 348, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.7/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.7/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


#Start Tensorboard

In [ ]:
LOG_DIR = '/drive/gpt-2-train/checkpoint/run1'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)


In [ ]:
!ps aux | grep tensorboard

root        1588  0.2  2.2 872976 304868 ?       Sl   10:00   0:06 /usr/bin/python2 /usr/local/bin/tensorboard --logdir /drive/gpt-2-train/checkpoint/run1 --host 0.0.0.0 --port 6006
root        2345  0.0  0.0  39192  6660 ?        S    10:32   0:00 bash /drive/ngrok-ssh/run_ngrok.sh ssh tensorboard tcp8080 tpu
root        2346  0.2  0.1 122768 19348 ?        Sl   10:32   0:01 /ngrok start --config /root/.ngrok2/ngrok.yml --config /drive/ngrok-ssh/ssh.yml --log=stdout --config /drive/ngrok-ssh/tensorboard.yml --config /drive/ngrok-ssh/http8080.yml --config /drive/ngrok-ssh/tcp8080.yml --config /drive/ngrok-ssh/tpu.yml ssh tensorboard tcp8080 tpu
root        2500  0.0  0.0  39192  6624 ?        S    10:39   0:00 /bin/bash -c ps aux | grep tensorboard
root        2502  0.0  0.0  38572  5684 ?        S    10:39   0:00 grep tensorboard


# How to view tensorboard

In [66]:
print('To view tensorboard, visit', get_ngrok_url('ssh'))

To view tensorboard, visit tcp://8.tcp.ngrok.io:12712


# Display each ngrok URL

For each ngrok tunnel, this cell prints its name, its public URL, and the local URL being forwarded to that public URL.

In [67]:
for name, tunnel in get_ngrok_tunnels():
  local_url = get_ngrok_url(name, local=True)
  public_url = get_ngrok_url(name, local=False)
  print('{:12s} {} <-> {}'.format(name, public_url, local_url))

ssh          tcp://8.tcp.ngrok.io:12712 <-> localhost:22


# How to SSH into this Colab instance

In [68]:
print('')
public_url, public_port = urllib.parse.urlparse(get_ngrok_url('ssh')).netloc.split(':')
print('To SSH into this colab instance, run the following command on your local machine:')
print('')

print('  ssh -p {} root@{}'.format(public_port, public_url))


To SSH into this colab instance, run the following command on your local machine:

  ssh -p 12712 root@8.tcp.ngrok.io
